<a href="https://colab.research.google.com/github/Alizah-cloud/AlizahAndCode/blob/main/Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install contractions

  Using cached contractions-0.1.73-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached textsearch-0.0.24-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached anyascii-0.3.2-py3-none-any.whl.metadata (1.5 kB)
  Using cached pyahocorasick-2.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (13 kB)
Using cached contractions-0.1.73-py2.py3-none-any.whl (8.7 kB)
Using cached textsearch-0.0.24-py2.py3-none-any.whl (7.6 kB)
Using cached anyascii-0.3.2-py3-none-any.whl (289 kB)
Using cached pyahocorasick-2.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (110 kB)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
reviews = pd.read_csv('IMDB Dataset.csv')

In [ ]:
reviews.isnull().values.any()

False

In [ ]:
reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Data Preprocessing

In [ ]:
def detect_contractions(text):
    contractions_dict = contractions.contractions_dict
    contraction_list = contractions_dict.keys()

    detected_contractions = [word for word in str(text).split() if word.lower() in contraction_list]
    return detected_contractions

reviews['contractions'] = reviews['review'].apply(detect_contractions)

In [ ]:
def expand_contractions_in_column(text):
    return contractions.fix(text)
reviews["Expanded_reviews"] = reviews["review"].apply(expand_contractions_in_column)

Removing: HTMl Tags, special character and stopwords & converting to lowercase

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    preprocessed_text = " ".join(words)
    return preprocessed_text

reviews['preprocessed_reviews'] = reviews['Expanded_reviews'].apply(preprocess_text)
print(reviews[['Expanded_reviews', 'preprocessed_reviews']].sample(5))

                                        Expanded_reviews  \
5332   Like many of you I am a great fan of the real ...   
14864  Ho, ho, homicidal maniac! This spirited tour-d...   
45476  We saw La Spettatrice last night @ the Chicago...   
26281  This independent, B&W, DV feature consistently...   
24474  Booted out of heaven, a gang of horny naked fe...   

                                    preprocessed_reviews  
5332   like many great fan real thing noir films red ...  
14864  ho ho homicidal maniac spirited tourdeforce ad...  
45476  saw la spettatrice last night chicago internat...  
26281  independent bw dv feature consistently shocks ...  
24474  booted heaven gang horny naked female angels b...  


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(reviews['sentiment'])

print(y)

[1 1 1 ... 0 0 0]


In [ ]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

X = [word_tokenize(sentence.lower()) for sentence in reviews['preprocessed_reviews']]
model = Word2Vec(sentences=X, vector_size=100, window=5, min_count=1, workers=4)

model.save("word2vec.model")
embedding_matrix = model.wv.vectors


In [ ]:
# Print the shape of the embedding matrix
print(f"Shape of embedding matrix: {embedding_matrix.shape}")

# Check the data type of the embedding matrix
print(f"Data type of embedding matrix: {embedding_matrix.dtype}")


Shape of embedding matrix: (213094, 100)
Data type of embedding matrix: float32


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming `X` is the feature matrix and `y` is the target vector
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, padding='post')
X_test_padded = pad_sequences(X_test_sequences, padding='post')


ModuleNotFoundError: No module named 'keras.preprocessing.text'

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense


model = Sequential()
model.add(Embedding(input_dim=213094, output_dim=100, weights=[embedding_matrix],trainable=False))

model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
trained_model = model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_data=(X_test_padded, y_test))

In [ ]:
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
import keras
model.save('sentiment_analysis_model.keras')